<a href="https://colab.research.google.com/github/yan-69/NLP/blob/master/Review_Sentiment_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

print("GPU is ", "on" if tf.config.list_physical_devices('GPU') else "off")


In [ ]:
dev_data , test_data = tfds.load(name = 'imdb_reviews' , split = [ "train" , "test"] , batch_size = -1 , as_supervised=True)

dev_examples , dev_label = tfds.as_numpy(dev_data)
test_examples , test_label = tfds.as_numpy(test_data)

print(dev_examples[:5])
print(dev_label[:5])

In [ ]:
print("dev_set contain {} examples and test_set contain {} examples" .format(len(dev_examples),len(test_examples)))

In [ ]:
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"


hub_layer = hub.KerasLayer(model , output_shape = [20] , input_shape =[] ,dtype = tf.string ,trainable = True )

hub_layer(dev_examples[:2])

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)))
model.add(tf.keras.layers.Dense(128 , activation ='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
    metrics =[tf.metrics.BinaryAccuracy(threshold = 0.0 , name ='accuracy')]
)

In [ ]:
val_examples = dev_examples[:10000]
dev_train = dev_examples[10000:]
val_labels = dev_label[:10000]
dev_train_labels = dev_label[10000:]

In [ ]:
history = model.fit(
    dev_train,dev_train_labels,batch_size = 512 ,epochs = 40 , verbose = 1 ,validation_data = (val_examples,val_labels)
)

In [ ]:
result = model.evaluate(test_examples,test_label)

print(result)

In [ ]:
history_dict = history.history
history_dict.keys()

In [ ]:

loss = history_dict['loss']
accuracy = history_dict['accuracy']
val_loss = history_dict['val_loss']
val_accuracy = history_dict['val_accuracy']

epochs = range(1,len(accuracy)+1)
plt.plot(epochs , loss , 'b' , label = 'Training loss')
plt.plot(epochs , val_loss , 'r' , label = 'Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()